In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import time

In [ ]:
import ct_projector.projector.numpy as ct_projector
import ct_projector.projector.numpy.fan_equiangluar as ct_fan

In [ ]:
# load a sample CT image
filename = './3.nii.gz'
ct = sitk.ReadImage(filename)
spacing = ct.GetSpacing()
img = sitk.GetArrayFromImage(ct)

# convert image from HU to attenuation coefficient
# This is the approximate relationship
img = (img.astype(np.float32) + 1024) / 1000 * 0.019
img[img < 0] = 0

# also convert to image to our projector dimension batch, z, y, x
img = img[np.newaxis,...]
img = img[:, ::-1, ...]
spacing = np.array(spacing[::-1])

In [ ]:
# show the ct images
plt.figure(figsize = (12,4))
plt.subplot(131); plt.imshow(img[0, img.shape[1]//2, ...], 'gray', aspect=spacing[1] / spacing[2])
plt.subplot(132); plt.imshow(img[0, :, img.shape[2]//2, :], 'gray', aspect=spacing[0] / spacing[2])
plt.subplot(133); plt.imshow(img[0, ..., img.shape[3]//2], 'gray', aspect=spacing[0] / spacing[1])

In [ ]:
# setup the projector
projector = ct_projector.ct_projector()
projector.from_file('./projector_fan.cfg')
projector.nx = img.shape[3]
projector.ny = img.shape[2]
projector.nz = img.shape[1]
projector.dx = spacing[2]
projector.dy = spacing[1]
projector.dz = spacing[0]
projector.nv = 64
projector.nview = 768

for k in vars(projector):
    print (k, '=', getattr(projector, k))

In [ ]:
# very important: use np.copy('C') so that the arrays are saved in C order
angles = projector.get_angles()
img = np.copy(img[..., :512, :512], 'C')
projector.set_projector(ct_fan.siddon_fp, angles = angles)

In [ ]:
# set which device to use
ct_projector.set_device(0)

# forward projection
start = time.time()
fp = projector.fp(img, angles = angles)
end = time.time()
print (end - start)


In [ ]:
fp.shape

In [ ]:
# show the projections
plt.figure(figsize = (8,8))
plt.imshow(fp[0,:, 25, :], 'gray')

In [ ]:
fprj = ct_fan.ramp_filter(projector, fp)

In [ ]:
plt.figure(figsize = (8,8))
plt.imshow(fprj[0,:, 25, :], 'gray')

In [ ]:
projector.set_backprojector(ct_fan.siddon_bp, angles = angles)
bp = projector.bp(fprj, angles = angles)

In [ ]:
plt.figure(figsize = (12,4))
plt.subplot(131); plt.imshow(bp[0, bp.shape[1]//2, ...], 'gray', aspect=spacing[1] / spacing[2])
plt.subplot(132); plt.imshow(bp[0, :, bp.shape[2]//2, :], 'gray', aspect=spacing[0] / spacing[2])
plt.subplot(133); plt.imshow(bp[0, ..., bp.shape[3]//2], 'gray', aspect=spacing[0] / spacing[1])

In [ ]:
fbp = ct_fan.fbp_bp(projector, fprj, angles = angles)

In [ ]:
plt.figure(figsize = (12,4))
plt.subplot(131); plt.imshow(fbp[0, fbp.shape[1]//2, ...], 'gray', aspect=spacing[1] / spacing[2])
plt.subplot(132); plt.imshow(fbp[0, :, fbp.shape[2]//2, :], 'gray', aspect=spacing[0] / spacing[2])
plt.subplot(133); plt.imshow(fbp[0, ..., fbp.shape[3]//2], 'gray', aspect=spacing[0] / spacing[1])